### Unpack all zip and rar files 

In [1]:
import os 
from os import path
import patoolib               #for unpacking
from pyunpack import Archive  # for unpacking
from shutil import copyfile
import pysrt
from mafan import simplify
from guess_language import guess_language
from itertools import compress

ImportError: No module named 'patoolib'

##### instructions for apt-get packages not available 
##### http://askubuntu.com/questions/283020/ubuntu-12-04-package-issues
##### follow https://github.com/ponty/pyunpack to install pyunpack

In [2]:
def check(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension == '.zip' or file_extension == '.rar' or file_extension == '.7z':
        return True
    else:
        return False
def check_srt(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension == '.srt':
        return True
    else:
        return False
def check_ass(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension == '.ass':
        return True
    else:
        return False
    
## use either gbk or big5 or utf-8
def open_srt(f):
    try:
        some_subs = pysrt.open(f,encoding = 'gbk') 
        return some_subs
    except:
        pass
    try:
        some_subs = pysrt.open(f,encoding = 'big5') 
        return some_subs
    except:
        pass
    try:
        some_subs = pysrt.open(f) 
        return some_subs
    except:
        pass
    
    return None

def validate(srt):
    subs = open_srt(srt) 
    if subs is not None:
        languages = [process_language(ele) for ele in subs]
        return languages
    else:
        #print('Can not decode.')
        return None
    
def process_language(ele):
    try:
        text_list = ele.text.split('\n')
        languages = [guess_language(l) for l in text_list]
        en_index = [l=='en' for l in languages]
        zh_index = [l=='zh' for l in languages]
        en_lines = list(compress(text_list,en_index))
        zh_lines = list(compress(text_list,zh_index))
        if len(en_lines)==0 or len(zh_lines)==0:
            return None
        en_text = ' '.join(en_lines)
        zh_text = simplify(' '.join(zh_lines))
        text_line = en_text + ' | ' + zh_text
        return text_line
    except:
        return None


In [3]:
os.getcwd()

'/home/johnsonice/Desktop/Ubuntu Backup/temp/webScrap/zimuku'

In [4]:
data_raw = 'data_raw/'
data_unpack = 'data_unpack/'
data_srt = 'data_srt/'
data_ass = 'data_ass/'
results_raw = 'data_results_raw/'

In [5]:
links = os.listdir(data_raw)
files = [f for f in links if check(f)]
len(files)

1227

In [6]:
## unpack all the data
for index,f in enumerate(files):
    try:
        f_name,f_ext = os.path.splitext(f)
        Archive(data_raw+f).extractall(data_unpack)
    except:
        pass

In [7]:
## get all srt files, including files in subfolders 
## get all ass files, influding files in sub folders 
unpack_links = []
unpack_names = []
for path, subdirs, files in os.walk(data_unpack):
    for name in files:
        unpack_links.append(os.path.join(path, name))
        unpack_names.append(name)
#unpack_links = os.listdir(data_unpack)
pair = zip(unpack_links,unpack_names)
asss = [f for f in pair if check_ass(f[0])]
pair = zip(unpack_links,unpack_names)
srts = [f for f in pair if check_srt(f[0])]
if len(srts)>0:
    [copyfile(f[0],data_srt+f[1]) for f in srts]
if len(asss)>0:
    [copyfile(f[0],data_ass+f[1]) for f in asss]

In [ ]:
## search for srts with both chinese and english in it
## and transform them into txt, seperated by |, dump to results raw
data_srts = [data_srt + f for f in os.listdir(data_srt)]
counter = 0 
for index, srt in enumerate(data_srts):
    res = validate(srt)
    if res is not None: 
        total = len(res)
        ## we only gonna to use files with both english and chinese in it 
        if total > 0:
            none_values = sum([x is None for x in res ])
            ratio = float(none_values)/float(total)
            if ratio > 0.5:  
                pass         
            else:
                res = res[5:-5]
                res = [x for x in res if x is not None]
                counter+=1
                filename, file_extension = os.path.splitext(srt)
                fname = filename.split('/')[-1]
                fname = results_raw + str(counter)+'_'+fname+'.txt'
                with open(fname, mode='wt', encoding='utf-8') as myfile:
                    myfile.write('\n'.join(res))
                #print(res)